# 1.Описание проекта и импорт библиотек.

В сформированном файле собраны данные о детских демисезонных ботинках для мальчиков и для девочек с 3 складов маркетплейса и отсортированы по популярности по состоянию на 18 марта 2024 года..

 Парсинг можно разделить на 2 условных этипа:

1. Сбор общей информации: бренд, артикул, размер, рейтинг, количество отзывов и цена.

2. Сбор подробных характеристик с карточки товара: материал верха, материал подкладки, высота подошвы и пр.


In [ ]:
import json
from pprint import pprint
import pandas as pd
import requests

# 2.Сбор общей информации о товарах.

Заготовим списки для будущих признаков собираемого датафрейма.

In [ ]:
ids = []                  # артикулы
brand = []                # бренд
brandId = []              # идентификатор бренда
reviewRating = []         # рейтинг товара
feedbacks = []            # отзывы, которые оставили покупатели
price = []                # цена

Чтобы собрать данные с отображаемой страницы пришлось "прокликать" страницы вручную, т.к. если номер страницы понятен, то параметр 'fdlvr' в опциях отследить не получилось. По времени обратока 10 страниц занимает около 12 минут.

Для формирования заголовков и параметров запроса я использовала следующий алгоритм: "Исследовать код" - "Network" - "Fetch/XHR" - находим файл json с данными каталога - "Copy as CURL(bash)". Переходим на сайт https://curlconverter.com/python/ , который полностью сформирует код запроса с заголовками и параметрами. И, если ответ сервера положителен, проводим сбор данных с json-файла.

Парсить страницы лучше по несколько штук и промежуточные результаты сохранять, т.к. сервер может выдавать ошибку 429 при усердной работе. Также нужно помнить, что каждая страница выдает по 100 позиций, и, т.к сайт динамический, в итоговом датафрейме будут накапливаться дубликаты.

Внизу пример кода для страницы под номером 1.

In [ ]:
headers = {
    'Accept': '*/*',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,de;q=0.6',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'DNT': '1',
    'Origin': 'https://www.wildberries.ru',
    'Pragma': 'no-cache',
    'Referer': 'https://www.wildberries.ru/catalog/obuv/detskaya?sort=popular&page=1&fseason=50',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'appType': '1',
    'cat': '128330',
    'curr': 'rub',
    'dest': '-1257786',
    'fseason': '50',
    'page': '1',
    'sort': 'popular',
    'spp': '30',
}

response = requests.get('https://catalog.wb.ru/catalog/children_shoes/v2/catalog', params=params, headers=headers)

# Сбор данных при положительном ответе.
if response.status_code == 200:
      data_js = json.loads(response.content)
      for i in range(100):
        ids.append(data_js["data"]["products"][i]['id'])
        brand.append(data_js["data"]["products"][i]['brand'])
        brandId.append(data_js["data"]["products"][i]['brandId'])
        reviewRating.append(data_js["data"]["products"][i]['reviewRating'])
        feedbacks.append(data_js["data"]["products"][i]['feedbacks'])
        price.append(data_js["data"]["products"][i]['sizes'][0]['price']['total'] * 0.01)

data = {'ids': ids,
        'brand': brand,
        'brandId': brandId,
        'reviewRating': reviewRating,
        'feedbacks': feedbacks,
        'price': price}

df = pd.DataFrame(data=data)

print(f'Найдено дублирующих строк: {df.duplicated().sum()}')
print('Размерность:', df.shape)
display(df.head())

Найдено дублирующих строк: 0
Размерность: (100, 6)


,ids,brand,brandId,reviewRating,feedbacks,price
0,146667605,Дюна,4272,4.8,3802,544.0
1,34774854,Biker,16825,4.9,58,779.0
2,151151496,Дюна,4272,4.8,3802,544.0
3,118988168,Demix,213234,4.8,683,2753.0
4,13199587,Biker,16825,4.9,237,779.0


In [ ]:
# Выгрузка результатов.
df.to_csv('catalog.csv', index=False)

# 3. Сбор подробных характеристик товара.

Для сбора более подробной информации о товаре нужно ознакомится с его описанием(карточкой). Карточка товара в Wildberries хранится в формате json, что делает сбор данных очень удобным. Выяснилось, что ссылка на карточку имеет 3 варианта шаблона (в зависимости от давности артикула):

1. Более половины наблюдений:  **https:// {basket-00.wbbasket.ru}
/vol {первые 4 символа артикула}
/part {первые 6 символов артикула} /{артикул}/info/ru/card.json**

2. Прочие варианты:  **https:// {basket-00.wbbasket.ru}
/vol {первые 3 символа артикула}
/part {первые 5 символов артикула} /{артикул}/info/ru/card.json** или **https:// {basket-00.wbbasket.ru}
/vol {первые 2 символа артикула}
/part {первые 4 символов артикула} /{артикул}/info/ru/card.json**

wbbasket.ru - сервис для зарегистрированных продавцов Wildberries. Значение "basket" может меняться, т.к. товары перемещаются между складами и продаются. В ходе исследования карточек товаров я заметила, что товары находящиеся на складах Алексин, Коледино и Подольск привязаны к номерам 03, 04, 05, 06, 09, 10, 12. В меньшей степени у basket были номера 13, 14, 08, 07. Для выявления ссылок на карточки товаров я решила сгенерировать возможные ссылки, из них отсортировать работающие. Работающие ссылки понадобятся для запросов напрямую в json-файл.

Загружаем сформированный ранее датафрейм, нам понадобятся артикулы для поиска. Пробуем сгенерировать ссылки.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving catalog.csv to catalog (1).csv


In [ ]:
df.duplicated().sum()

0

Список артикулов преобразуем во множество для избавления от дубликатов, если они появились.

In [ ]:
ids_list = list(set(df.ids))

basket_numbers = ['basket-09.wbbasket.ru',
                  'basket-12.wbbasket.ru',
                  'basket-03.wbbasket.ru',
                  'basket-04.wbbasket.ru',
                  'basket-05.wbbasket.ru',
                  'basket-10.wbbasket.ru',
                  'basket-06.wbbasket.ru',
                  'basket-13.wbbasket.ru',
                  'basket-14.wbbasket.ru',
                  'basket-08.wbbasket.ru',
                  'basket-07.wbbasket.ru'
                  ]

def links_maker(ids_list):
  "Функция, которая генерирует возможные ссылки по списку артикулов (ids_list)"
  cards_links_list = []
  for id in ids_list:
    str_id = str(id)
    for basket in basket_numbers:
      cards_links_list.append(f'https://{basket}/vol{str_id[:4]}/part{str_id[:6]}/{str_id}/info/ru/card.json')
  return cards_links_list


cards_links_list = links_maker(ids_list)
cards_links_list[:7]
print("Количество ссылок для обработки:", len(cards_links_list))

Количество ссылок для обработки: 1100


"Прокликаем" сгенерированные ссылки. 1000 ссылок обрабатывается за 10 минут. Если пропущенных артикулов осталось немного (до 50), то быстрее будет прокликать их вручную. Здесь также рекомендуется периодически сохранять промежуточные результаты, т.к. работы много, у меня заняло около 7 итераций.

In [ ]:
# Проверяем ссылки на работоспособность

def check_link(cards_links_list):
  '''Функция принимает список ссылок(cards_links_list) и собирает
  неработающие ссылки в отдельный лист: for_delete'''
  for_delete = []
  for link in cards_links_list:
    response = requests.get(link)
    if response.status_code == 404:
      for_delete.append(link)
  return for_delete

for_delete_end = check_link(cards_links_list)

In [ ]:
cards_links_list = set(cards_links_list) - set(for_delete_end)
print("Количество ссылок после обработки:", len(cards_links_list))

# Преобразование в список для дальнейшей работы
cards_links = list(cards_links_list)

Количество ссылок после обработки: 52


Извлекаем полезную информацию из ссылок:
  * артикул товара;
  * описание;
  * цвет;
  * цвет;
  * состав;
  * материал подошвы обуви;
  * материал стельки;
  * высота подошвы;
  * материал подкладки обуви;
  * страна производства.


In [ ]:
def data_extraction(link):
  '''Функция принимает на вход список ссылку (link)
  и извлекает информацию'''

  cards_data = {'ids': "",
                "description": "",
                'color': "",
                'compound': "",
                'sole_material': "",
                'insole': "",
                'sole_heighte': "",
                'shoes_lining': "",
                'country': ""

  }
  response = requests.get(link)

  if response.status_code == 200:
    card_json = json.loads(response.text)
    cards_data['ids'] = card_json['nm_id']

    if "description" in card_json:
      cards_data['description'] = card_json["description"]

    for i in range(len(card_json['options'])):

      if card_json['options'][i]['name'] == "Цвет":
        cards_data['color'] = card_json['options'][i]['value']

      if card_json['options'][i]['name'] == "Состав":
        cards_data['compound'] = card_json['options'][i]['value']

      if card_json['options'][i]['name'] == "Материал подошвы обуви":
        cards_data['sole_material'] = card_json['options'][i]['value']

      if card_json['options'][i]['name'] == "Материал стельки":
        cards_data['insole'] = card_json['options'][i]['value']

      if card_json['options'][i]['name'] == "Высота подошвы":
        cards_data['sole_heighte'] = card_json['options'][i]['value']

      if card_json['options'][i]['name'] == "Материал подкладки обуви":
        cards_data['shoes_lining'] = card_json['options'][i]['value']

      if card_json['options'][i]['name'] == "Страна производства":
        cards_data['country'] = card_json['options'][i]['value']

  return  cards_data

Формируем датафрейм и выгружаем результат, если нужно.

In [ ]:
df_cards = pd.DataFrame(columns=['ids', 'description', 'color', 'compound',
                           'sole_material','shoes_lining', 'insole',
                           'sole_heighte', 'country'])

for link, i in zip(cards_links, range(len(cards_links))):
  df_cards.loc[i] = data_extraction(link)

#df_cards.to_csv('data_cards.csv', index=False)

df_cards.head()

,ids,description,color,compound,sole_material,shoes_lining,insole,sole_heighte,country
0,149523531,"Уже давно не секрет, что юные модницы стараютс...",черный,искусственная лаковая кожа; искусственная кожа,полиуретан,хлопок; полиэстер,хлопок; полиэстер,2.4 см,Китай
1,118988199,Кроссовки Like от Demix — идеальный выбор для ...,белый,полиэстер; полиуретан,ЭВА,полиэстер,искусственный материал,,Китай
2,167964881,Стильные ботинки детские бренда TimeJump униве...,темно-синий,Ткань; искусственная кожа; ТПУ,ЭВА,хлопок,хлопок,,Китай
3,110720640,Не спешите писать негативный отзыв при следующ...,темно-синий,искусственная кожа; искусственная лаковая кожа...,ТПР,хлопок,хлопок,,Китай
4,169572827,Стильные ботинки детские бренда Biker универса...,черный,искусственный нубук; искусственная кожа; резина,ТПР,хлопок,хлопок,,Китай


# 4. Сборка итогового датафрейма.

Соединим данные 2 таблиц по признаку 'ids' и сохраняем готовый результат.

In [ ]:
merged = df.merge(df_cards, how='right', on='ids')
merged.to_csv('Wildberries_data.csv', index=False)

merged.head()

,ids,brand,brandId,reviewRating,feedbacks,price,description,color,compound,sole_material,shoes_lining,insole,sole_heighte,country
0,149523531,T.TACCARDI,16819,5.0,50,779.0,"Уже давно не секрет, что юные модницы стараютс...",черный,искусственная лаковая кожа; искусственная кожа,полиуретан,хлопок; полиэстер,хлопок; полиэстер,2.4 см,Китай
1,118988199,Demix,213234,4.9,249,2753.0,Кроссовки Like от Demix — идеальный выбор для ...,белый,полиэстер; полиуретан,ЭВА,полиэстер,искусственный материал,,Китай
2,167964881,TimeJump,16818,4.4,32,779.0,Стильные ботинки детские бренда TimeJump униве...,темно-синий,Ткань; искусственная кожа; ТПУ,ЭВА,хлопок,хлопок,,Китай
3,110720640,Honey girl,16824,4.7,96,779.0,Не спешите писать негативный отзыв при следующ...,темно-синий,искусственная кожа; искусственная лаковая кожа...,ТПР,хлопок,хлопок,,Китай
4,169572827,Biker,16825,4.9,171,1112.0,Стильные ботинки детские бренда Biker универса...,черный,искусственный нубук; искусственная кожа; резина,ТПР,хлопок,хлопок,,Китай
